In [ ]:
# this notebook was created during LLM zoomcamp 2024
# it's for HW 1

In [ ]:
# Q1
# !pip install elasticsearch
from elasticsearch import Elasticsearch

In [ ]:
# create elastic search client
es_client = Elasticsearch('http://localhost:9200')
# check that it worked and retrieve info for hw1 Q1
Q1 = es_client.info()['version']['build_hash']

In [ ]:
# get data and index

In [ ]:
!pip install requests

In [ ]:
# get FAQ for HW
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
# Use only question and text fields and give question a boost of 4, and use "type": "best_fields".
# for Q3 _score needed

In [ ]:
# index data
# create an index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # we use course names for filtering
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index = index_name, body = index_settings)

In [ ]:
# index the docs
for doc in documents:
    es_client.index(index = index_name, document = doc)

In [ ]:
Q2 = "index"

In [ ]:
# modify function as stated in the task and to return more details (we need _score)
def elastic_search_score(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], # ^4 - makes this field 4x important
                        "type": "best_fields"
                    }
                # },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp" # it wasn't specified in the task
                #     }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body = search_query)
    # result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return response

In [ ]:
# our new query
query = "How do I execute a command in a running docker container?"

In [ ]:
# Q3 get score of the best answer
Q3 = elastic_search_score(query)['hits']['max_score']

In [ ]:
# Q4. limit the questions to machine-learning-zoomcamp, return 3 results
def elastic_search(query, course):
    search_query = {
        "size": 3, # asked for 3 results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], # ^4 - makes this field 4x important
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course # use a variable
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    # result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return response

In [ ]:
query = "How do I execute a command in a running docker container?"
course = "machine-learning-zoomcamp"

# Q4 answer
Q4 = elastic_search(query, course)['hits']['hits'][2]['_source']['question']

In [ ]:
# Q5. return question and text for the prompt
def elastic_search_qt(query, course):
    search_query = {
        "size": 3, # asked for 3 results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"], # ^4 - makes this field 4x important
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course # use a variable
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    question = []
    
    for hit in response['hits']['hits']:
        question.append(hit['_source']['question'])
    
    text = []

    for hit in response['hits']['hits']:
        text.append(hit['_source']['text'])
        
    return text, question

In [ ]:
# test
response_t, response_q = elastic_search_qt(query, course)
response_t, response_q

In [ ]:
# build a prompt
# define a function for building a prompt using given templates
def build_prompt(query, text, question):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = context_template.format(question = question, text = text).strip()

    prompt = prompt_template.format(question = query, context = context).strip()
    return prompt

In [ ]:
#Q5 calculate len
response_t, response_q = elastic_search_qt(query, course)
Q5 = len(build_prompt(query, response_t, response_q))

In [ ]:
our_prompt = build_prompt(query, response_t, response_q)

In [ ]:
# tokens
!pip install tiktoken

In [ ]:
# count tokens
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
Q6 = len(encoding.encode(our_prompt))